In [1]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split, cross_validate, KFold

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import lightgbm as lgb

from sklearn.metrics import accuracy_score


import optuna


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
Трейн и тест предобработаны следующим образом:

from textblob import TextBlob
from textblob import Word

import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')


df['text'] = df.ingredients.apply(lambda x: ' '.join(x))

df['tokens'] = df.text.apply(lambda i: ' '.join([Word(x).lemmatize()
                            for x in TextBlob(i).lower().words if Word(x).isalpha() == True and x not in stopwords]))
'''

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Проект/pretrain.csv')
df

,id,cuisine,ingredients,text,tokens
0,10259,greek,"['romaine lettuce', 'black olives', 'grape tom...",romaine lettuce black olives grape tomatoes ga...,romaine lettuce black olive grape tomato garli...
1,25693,southern_us,"['plain flour', 'ground pepper', 'salt', 'toma...",plain flour ground pepper salt tomatoes ground...,plain flour ground pepper salt tomato ground b...
2,20130,filipino,"['eggs', 'pepper', 'salt', 'mayonaise', 'cooki...",eggs pepper salt mayonaise cooking oil green c...,egg pepper salt mayonaise cooking oil green ch...
3,22213,indian,"['water', 'vegetable oil', 'wheat', 'salt']",water vegetable oil wheat salt,water vegetable oil wheat salt
4,13162,indian,"['black pepper', 'shallots', 'cornflour', 'cay...",black pepper shallots cornflour cayenne pepper...,black pepper shallot cornflour cayenne pepper ...
...,...,...,...,...,...
39769,29109,irish,"['light brown sugar', 'granulated sugar', 'but...",light brown sugar granulated sugar butter warm...,light brown sugar granulated sugar butter warm...
39770,11462,italian,"['KRAFT Zesty Italian Dressing', 'purple onion...",KRAFT Zesty Italian Dressing purple onion broc...,kraft zesty italian dressing purple onion broc...
39771,2238,irish,"['eggs', 'citrus fruit', 'raisins', 'sourdough...",eggs citrus fruit raisins sourdough starter fl...,egg citrus fruit raisin sourdough starter flou...
39772,41882,chinese,"['boneless chicken skinless thigh', 'minced ga...",boneless chicken skinless thigh minced garlic ...,boneless chicken skinless thigh minced garlic ...


In [5]:
X = df.tokens
y = df.cuisine

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25)

In [ ]:
# Optuna для подбора гиперпараметров всех моделей

def objective_p(trial): 
    
    #C = trial.suggest_float('C', 1e-5, 5, log=True)
    
    #min_samples_split = trial.suggest_int('min_samples_split', 2, 200)
    #min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 100)
    #max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 20, 20000)
    
    n_estimators = trial.suggest_int('n_estimators', 500, 2000)
    #max_features = trial.suggest_categorical('max_features', ["sqrt", "log2"])
    #max_samples = trial.suggest_float('max_samples', 0.05, 1)
    #oob_score = trial.suggest_categorical('oob_score', [True, False])
    #criterion = trial.suggest_categorical('criterion', ["gini", "entropy", "log_loss"])

    reg_alpha = trial.suggest_float('reg_alpha', 1e-5, 0.2, log=True)
    reg_lambda = trial.suggest_float('reg_alpha', 1e-5, 0.2, log=True)
    max_depth = trial.suggest_int('max_depth', 1, 50)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 0.1, log=True)

    

    p = Pipeline([
    ('nlp', TfidfVectorizer(
                    min_df=2,
                    max_df=0.5,
                    #max_features=100,
                    #ngram_range=(1, 3)
                    stop_words='english'
                   )),
    
    #('dimred', SelectKBest(k=100)),
    #('dimred', SelectPercentile(percentile=10)),
    
    #('model', LogisticRegression(C=C, max_iter=1000))

    #('model', DecisionTreeClassifier(min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_leaf_nodes=max_leaf_nodes))

    #('model', RandomForestClassifier(n_jobs=-1,
        #min_samples_split=min_samples_split,
        #min_samples_leaf=min_samples_leaf,
        #max_leaf_nodes=max_leaf_nodes,
        
        #n_estimators=n_estimators,
        #max_features=max_features,
        #max_samples=max_samples,
        
        #oob_score=oob_score,
        #criterion=criterion,
    #))

    #('model', GradientBoostingClassifier())

    ('model', lgb.LGBMClassifier(n_jobs=-1, 
                                 n_estimators=n_estimators,
                                 reg_alpha=reg_alpha,
                                 reg_lambda=reg_lambda,
                                 max_depth=max_depth,
                                 learning_rate=learning_rate       
    ))

    ])
    
    
    cv = cross_validate(p, X, y, cv=3, scoring='accuracy', n_jobs=-1, return_train_score=True)
    score = cv['test_score'].mean()

    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective_p, n_trials=30)

[I 2023-05-22 19:29:50,086] A new study created in memory with name: no-name-714f91ef-2e0a-4218-992f-675035dd950f
[W 2023-05-22 19:36:29,209] Trial 0 failed with parameters: {'n_estimators': 953, 'reg_alpha': 1.3528511160887914e-05, 'max_depth': 48, 'learning_rate': 0.03167224617872526} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-36-bdb464c87b60>", line 66, in objective_p
    cv = cross_validate(p, X, y, cv=3, scoring='accuracy', n_jobs=-1, return_train_score=True)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 266, in cross_validate
    results = parallel(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 63, in __call__
    return super().__call__(iterable_with_config)
  File "/usr/local/lib/python3.10/dist-pa

KeyboardInterrupt: ignored

In [ ]:
# Pipeline с подобранными параметрами

p = Pipeline([
    ('nlp', TfidfVectorizer(
                    min_df=2,
                    max_df=0.5,
                    #max_features=100,
                    #ngram_range=(1, 3)
                    stop_words='english'
                   )),
    
    
    #('dimred', SelectKBest(k=100)),
    #('dimred', SelectPercentile(percentile=10)),
    
   
    #('model', LogisticRegression(
        #C=2.7056825281253234, 
        #))
    
    #('model', DecisionTreeClassifier(
        #min_samples_split=65, 
        #min_samples_leaf=7, 
        #max_leaf_nodes=8018
    #))
    
    #('model', RandomForestClassifier(
        #min_samples_split=65,
        #min_samples_leaf=7,
        #max_leaf_nodes=8018
    #))
    
    #('model', GradientBoostingClassifier())

    ('model', lgb.LGBMClassifier(n_jobs=-1, 
                                 n_estimators=279,
                                 max_depth=22,
                                 learning_rate=0.03803766047009802       
    ))

    ])

In [ ]:
%%time
p.fit(Xtrain, ytrain)

CPU times: user 5min 16s, sys: 1.12 s, total: 5min 17s
Wall time: 3min 22s


Pipeline(steps=[('nlp',
                 TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')),
                ('model',
                 LGBMClassifier(learning_rate=0.03803766047009802, max_depth=22,
                                n_estimators=279))])

In [ ]:
trainpredict = p.predict(Xtrain)
predict = p.predict(Xtest)

accuracy_score(ytrain, trainpredict), accuracy_score(ytest, predict)

(0.982031511900771, 0.7934432823813354)

In [ ]:
cv = cross_validate(p, X, y, cv=3, scoring='accuracy', n_jobs=-1, return_train_score=True)
cv

{'fit_time': array([5.37124634, 5.35618019, 4.48733521]),
 'score_time': array([0.37369323, 0.36056519, 0.22114468]),
 'test_score': array([0.78435661, 0.78397948, 0.78322522]),
 'train_score': array([0.8431513 , 0.84088852, 0.84145422])}

In [ ]:
round(cv['train_score'].mean(), 5), round(cv['test_score'].mean(), 5)

(0.84183, 0.78385)

In [ ]:
# submit

p.fit(X, y)
submit = pd.read_csv('/content/drive/MyDrive/Проект/presubmit.csv')
Xsubmit = submit.tokens
predict = p.predict(Xsubmit)
submit = pd.concat([submit[['id']], pd.DataFrame(predict, columns=['cuisine'])], axis=1)
submit.to_csv('/content/drive/MyDrive/Проект/submit.csv', index=False)

#### Отсюда идёт гибрид

In [9]:
p_logit = Pipeline([
    ('nlp', TfidfVectorizer(
                    min_df=2,
                    max_df=0.5,
                    stop_words='english'
                   )),
    ('model', LogisticRegression(n_jobs=-1,
        C=2.7056825281253234
        ))
    ])

In [10]:
p_LGBM = Pipeline([
    ('nlp', TfidfVectorizer(
                    min_df=2,
                    max_df=0.5,
                    stop_words='english'
                   )),
    ('model', lgb.LGBMClassifier(n_jobs=-1, 
                                 n_estimators=279,
                                 max_depth=22,
                                 learning_rate=0.03803766047009802
    ))  
    ])

In [15]:
# Optuna для подбора ratio

def objective_p(trial): 
    
    #C = trial.suggest_float('C', 1e-5, 5, log=True)
    
    #n_estimators = trial.suggest_int('n_estimators', 1, 500)
    #max_depth = trial.suggest_int('max_depth', 1, 50)
    #learning_rate = trial.suggest_float('learning_rate', 1e-5, 0.1, log=True)

    ratio = trial.suggest_float('ratio', 0, 1)

    
    p_logit = Pipeline([
    ('nlp', TfidfVectorizer(
                    min_df=2,
                    max_df=0.5,
                    stop_words='english'
                   )),
    ('model', LogisticRegression(n_jobs=-1,
        C=2.7056825281253234
        ))
    ])

    p_LGBM = Pipeline([
    ('nlp', TfidfVectorizer(
                    min_df=2,
                    max_df=0.5,
                    stop_words='english'
                   )),
    ('model', lgb.LGBMClassifier(n_jobs=-1, 
                                 n_estimators=279,
                                 max_depth=22,
                                 learning_rate=0.03803766047009802
    ))  
    ])
    
    
    kf = KFold(n_splits=3, shuffle=True)
    kf.split(X, y)

    accuracy = []
    for train_index, test_index in kf.split(X):

      Xtrain = X[train_index]
      Xtest = X[test_index]
      ytrain = y[train_index]
      ytest = y[test_index]

      p_logit.fit(Xtrain, ytrain)
      p_LGBM.fit(Xtrain, ytrain)

      proba_logit = p_logit.predict_proba(Xtest)
      proba_LGBM = p_LGBM.predict_proba(Xtest)

      ratio = ratio

      proba_hybrid = proba_logit * ratio + proba_LGBM * (1 - ratio)
      predict_hybrid = np.array([p_logit.classes_[proba_hybrid[i].argmax()] for i in range(proba_hybrid.shape[0])])

      accuracy.append(accuracy_score(ytest, predict_hybrid))
   
    score = np.array(accuracy).mean()

    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective_p, n_trials=20)

[I 2023-05-23 08:32:55,975] A new study created in memory with name: no-name-2d527698-ceb5-4216-bad7-f92669132a8a
[I 2023-05-23 08:46:31,000] Trial 0 finished with value: 0.7964248001206817 and parameters: {'ratio': 0.650161866852695}. Best is trial 0 with value: 0.7964248001206817.
[I 2023-05-23 08:58:53,059] Trial 1 finished with value: 0.7853622969779251 and parameters: {'ratio': 0.9630918889445321}. Best is trial 0 with value: 0.7964248001206817.
[I 2023-05-23 09:11:02,154] Trial 2 finished with value: 0.7944385779655051 and parameters: {'ratio': 0.7491160424578214}. Best is trial 0 with value: 0.7964248001206817.
[I 2023-05-23 09:23:13,898] Trial 3 finished with value: 0.795720822647961 and parameters: {'ratio': 0.5532268971607192}. Best is trial 0 with value: 0.7964248001206817.
[I 2023-05-23 09:35:34,421] Trial 4 finished with value: 0.7962488057525016 and parameters: {'ratio': 0.6117397022066663}. Best is trial 0 with value: 0.7964248001206817.
[I 2023-05-23 09:47:51,982] Trial

In [ ]:
# Fit, predict

%%time

p_logit.fit(Xtrain, ytrain)
p_LGBM.fit(Xtrain, ytrain)

proba_logit_train = p_logit.predict_proba(Xtrain)
proba_LGBM_train = p_LGBM.predict_proba(Xtrain)

proba_logit = p_logit.predict_proba(Xtest)
proba_LGBM = p_LGBM.predict_proba(Xtest)

ratio = 0.5067968782043537

proba_hybrid_train = proba_logit_train * ratio + proba_LGBM_train * (1 - ratio)
predict_hybrid_train = np.array([p_logit.classes_[proba_hybrid_train[i].argmax()] for i in range(proba_hybrid_train.shape[0])])

proba_hybrid = proba_logit * ratio + proba_LGBM * (1 - ratio)
predict_hybrid = np.array([p_logit.classes_[proba_hybrid[i].argmax()] for i in range(proba_hybrid.shape[0])])

round(accuracy_score(ytrain, predict_hybrid_train), 5), round(accuracy_score(ytest, predict_hybrid), 5)

In [17]:
# CV

kf = KFold(n_splits=3, shuffle=True)
kf.split(X, y)

accuracy_train = []
accuracy = []
for train_index, test_index in kf.split(X):

  Xtrain = X[train_index]
  Xtest = X[test_index]
  ytrain = y[train_index]
  ytest = y[test_index]

  p_logit.fit(Xtrain, ytrain)
  p_LGBM.fit(Xtrain, ytrain)

  proba_logit_train = p_logit.predict_proba(Xtrain)
  proba_LGBM_train = p_LGBM.predict_proba(Xtrain)
  
  proba_logit = p_logit.predict_proba(Xtest)
  proba_LGBM = p_LGBM.predict_proba(Xtest)

  ratio = 0.5067968782043537

  proba_hybrid_train = proba_logit_train * ratio + proba_LGBM_train * (1 - ratio)
  predict_hybrid_train = np.array([p_logit.classes_[proba_hybrid_train[i].argmax()] for i in range(proba_hybrid_train.shape[0])])
  
  proba_hybrid = proba_logit * ratio + proba_LGBM * (1 - ratio)
  predict_hybrid = np.array([p_logit.classes_[proba_hybrid[i].argmax()] for i in range(proba_hybrid.shape[0])])

  accuracy_train.append(accuracy_score(ytrain, predict_hybrid_train))
  accuracy.append(accuracy_score(ytest, predict_hybrid))

  
round(np.array(accuracy_train).mean(), 5), round(np.array(accuracy).mean(), 5)

(0.95564, 0.79539)

In [18]:
# submit_hybrid

p_logit.fit(X, y)
p_LGBM.fit(X, y)

submit = pd.read_csv('/content/drive/MyDrive/Проект/presubmit.csv')
Xsubmit = submit.tokens

proba_logit = p_logit.predict_proba(Xsubmit)
proba_LGBM = p_LGBM.predict_proba(Xsubmit)

ratio = 0.5
proba_hybrid = proba_logit * ratio + proba_LGBM * (1 - ratio)
predict_hybrid = np.array([p_logit.classes_[proba_hybrid[i].argmax()] for i in range(proba_hybrid.shape[0])])

submit = pd.concat([submit[['id']], pd.DataFrame(predict_hybrid, columns=['cuisine'])], axis=1)
submit.to_csv('/content/drive/MyDrive/Проект/submit.csv', index=False)